In [38]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import json

agents = {'BayesAgent':(r'AI$\xi$','purple'),
          'Knowledge-seeking agent':('Kullback-Leibler','blue'),
          'KullbackLeiblerKSA':('Kullback-Leibler','blue'),
          'ShannonKSA':('Shannon','green'),
          'SquareKSA':('Square','red'),
          'Shannon KSA':('Shannon','green'),
          'Square KSA':('Square','red'),
          'ThompsonAgent':('Thompson','yellow'),
          'QLearn':('Q-Learning','black'),
          'KSA-Dirichlet': ('Kullback-Leibler','blue'),
          'Entropy-seeking agent': ('Shannon','green'),
          'Square KSA-Dirichlet': ('Square','red')}


def plot_results(directory,filename='results-1',objective=None,outfile=None):
    if not objective:
        if 'ksa' in directory:
            objective = 'explored'
        else:
            objective = 'rewards'
    if objective == 'rewards':
        y_axis = 'Average Reward'
    elif objective == 'explored':
        y_axis = 'Exploration (%)'
    
    file = open(directory + '/' + filename + '.json')
    data = json.load(file)
    file.close()

    fig = plt.figure(figsize=(12,8),dpi=200)
    for i,k in enumerate(data):
        try:
            d = data[k]
        except KeyError:
            continue

        cycles = d[0]['cycles']
        runs = len(d)

        A = np.zeros((cycles,runs))
        for j in range(runs):
            A[:,j] = np.array(d[j][objective])
        mu = np.mean(A,1)
        sigma = np.std(A,1)
        a = np.max(np.vstack((mu-sigma,np.min(A,1))),0)
        b = np.min(np.vstack((mu+sigma,np.array(cycles*[100]))),0)

        if k in agents:
            lab = agents[k][0]

        color = agents[k][1]
        alpha = 0.2
        plt.plot(a,color=color,alpha=alpha)
        plt.plot(b,color=color,alpha=alpha)
        plt.plot(mu,label=lab,color=color,lw=3)
        plt.fill_between(np.arange(cycles),a,b,alpha=alpha,color=color)

    plt.xlabel('Cycles',fontsize=25)
    plt.ylabel(y_axis,fontsize=25)
    plt.legend(loc='lower right',fontsize=25)
    
    if outfile:
        plt.savefig(directory + '/' + outfile + '.png', bbox_inches='tight')
        plt.close()
        
plot_results('ksa',outfile='plot-paper')